<a href="https://colab.research.google.com/github/swoo-nam/project_final_team1/blob/main/%EB%82%A8%EC%83%81%EC%9A%B0_%EB%8D%B0%EC%9D%B4%ED%84%B0%EC%A0%84%EC%B2%98%EB%A6%AC_gpt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [121]:
# !pip install kiwipiepy

In [122]:
# !pip install konlpy

In [123]:
# !pip install transformers

In [124]:
# !pip install optuna

In [125]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [126]:
import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
from torch.utils.data.dataset import Dataset

from sklearn.utils import resample

In [127]:
import warnings
%config InlineBackend.figure_format = 'retina'
warnings.filterwarnings("ignore")
%matplotlib inline

In [128]:
import pandas as pd
import numpy as np
import re
from tqdm.auto import tqdm
import random
import os

def reset_seeds(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

DATA_PATH = "/content/drive/MyDrive/project_final/data/"
# DATA_PATH = "C:\python-code\project_final\data/"
SEED = 42

device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cpu'

# 데이터 불러오기

In [129]:
gpt_neg = pd.read_csv(f"{DATA_PATH}gpt_neg_reviews_4400.csv")
gpt_neg

,Unnamed: 0,original_review,original_reply,rating,gpt_review,gpt_reply,sentiment,score
0,0,맛도 양도 모두 만족합니다 배달도 정확히 보내 주시고 아주 만족합니다.,"안녕하세요! 주문해 주셔서 진심으로 감사드려요! 이렇게 좋은 리뷰 써 주시니, 메뉴...",5,김치찌개는 별로였어요. 약간 김치국 느낌이 나서 실망했어요. 고기도 별로였고.,안녕하세요. 김치찌개와 고기에 대한 실망을 드려 죄송합니다. 지속적인 품질 개선을 ...,부정,0.908812
1,1,너무 맛있게 잘 먹었습니다!,즐거운 순간을 저희와 함께해주셔서 감사합니다 보람차게 일할 수 있게 해 주셔서 감사...,5,양이 너무 많아서 배터질뻔했어요. 서비스도 별로였어요.,배터질뻔하셨다니 정말 안타깝습니다. 양에 대한 조정을 더 신경써야겠습니다. 불편을 ...,부정,0.820514
2,2,너무 배고파서 먹다가 찍었어요. 너무맛있어요,행복한 하루를 만들어 주셔서 감사합니다 직접 드시는 건데 맛은 당연히 좋아야 된다고...,5,음식이 너무 싱겁고 부족해서 맛이 없었어요. 배달도 늦게 왔고 실망스러웠어요.,음식 맛과 배달 시간에 대한 실망을 드려 죄송합니다. 앞으로 더 신속하고 맛있는 음...,부정,0.941290
3,3,두번째시켜요~ 이번엔 아이들이 잡채가 먹고싶대서 주문해봤는데 고기도 많이 들어가있고...,다시 주문해 주셔서 감사합니다 저희 메뉴 맛있죠? 이제 점점 저희 메뉴만 생각 나실...,5,잡채는 짜다고 하지도 않았는데 맛이 진짜 짜요. 먹기 어려웠어요.,잡채가 짜다는 피드백을 반영하지 못해 죄송합니다. 더 맛있고 조리하기 쉽도록 개선하...,부정,0.910234
4,4,푸짐하고 너무 맛있어용~,이렇게 저희를 선택해 주셔서 감사합니다~ ~ 맛있게 먹다가 양이 부족하다.? 저는 ...,5,가격에 비해 음식의 양도 적고 맛도 별로였어요. 실망이었습니다.,가격과 맛에 대한 실망을 드려 죄송합니다. 더 다양한 메뉴와 향상된 품질로 보답하겠...,부정,0.928585
...,...,...,...,...,...,...,...,...
4396,5335,너무 맛있게 잘먹었습니다! 일주일에 세번씩은 꼭 시키는것 같아요~!,앞으로도 초밥이 생각나실 땐 부성초밥 찾아주세요~ 항상 신선하고 맛있는 음식 보내...,5,서비스도 좋지 않았고 음식도 맛이 별로였어요. 실망했습니다,서비스와 음식의 맛 모두 아쉽게 느껴져 죄송합니다. 향후에는 만족할 수 있는 서비...,부정,0.931082
4397,5336,단골입니다 일전에 닉네임 nangja 을 안 썼더니 사장님의 요청이 맛있게 먹고 냉...,고객님의 리뷰가 저희에겐 크나 큰 힘이 된답니다 언제나 만족스러운 식사시간 되실 ...,5,배달이 많이 늦어서 실망했어요. 음식도 그다지 맛있지 않았어요,늦은 배달로 인해 실망을 드렸군요. 더 신속한 배달을 위해 노력하겠습니다. 또한 ...,부정,0.932665
4398,5337,처음 주문 했는데. 너무 맛있어요. 특히 VIP 초밥은 고퀄재료들로 다양하게 구성되...,식사가 만족스러우셨다니 너무 기분이 좋습니다 항상 고객님을 생각하며 신선한 음식만 ...,5,여러 번 시켜먹었는데 이번에는 별로였어요. 왠지 점점 맛이 없어지는 것 같아요,연이어 아쉬운 경험을 드려 정말 죄송합니다. 계속해서 음식의 맛을 향상시킬 수 있...,부정,0.890285
4399,5338,빠르게 잘 받았습니다 좋아하는 초밥이 다 있네요~ 배달은 딱 먹고 싶은 초밥구성이 ...,"주문해주셔서 정말 감사드립니다 저희 부성초밥은 매순간 최선을 다 하며, 개선해야 할...",5,음식이 맛이 없었어요. 다른 곳으로 시켜서 먹어야겠어요,맛이 없다니 정말 죄송합니다. 다른 가게의 음식을 주문하셔서 만족스러운 식사를 하...,부정,0.921841


In [130]:
gpt_neg['gpt_reply'][4399]

' 맛이 없다니 정말 죄송합니다. 다른 가게의 음식을 주문하셔서 만족스러운 식사를 하실 수 있도록 도움을 드리겠습니다. , '

In [131]:
print(gpt_neg[gpt_neg['gpt_reply'].str.contains("사장님의 답변이 없습니다 ,")].index)

Int64Index([3693, 3694, 3695, 3696, 3697, 3698, 3699, 3700, 3701, 3702], dtype='int64')


In [132]:
gpt_neg['gpt_review'][3698]

' 지금껏 먹은 닭볶음탕중에 최고입니다~ 정말 맛있네요 자주 시켜먹을께요 맛이 별로였어요, 음식이 촉촉하지 않고 식어서 먹기가 좀 그랬네요. 다음에는 안시킬 거 같아요 '

In [133]:
gpt_neg['gpt_reply'][3698]

' 사장님의 답변이 없습니다 , '

In [134]:
print(gpt_neg[gpt_neg['gpt_review'].str.contains("지금껏 먹은 닭볶음탕중에 최고입니다")].index)

Int64Index([3698], dtype='int64')


In [135]:
# 결측치 확인
gpt_neg.isnull().sum()

Unnamed: 0         0
original_review    0
original_reply     0
rating             0
gpt_review         0
gpt_reply          0
sentiment          0
score              0
dtype: int64

In [136]:
# 긍정 데이터 확인
gpt_neg[gpt_neg['sentiment'] == '긍정']

,Unnamed: 0,original_review,original_reply,rating,gpt_review,gpt_reply,sentiment,score


In [137]:
# 부정 -> 0 으로 치환
gpt_neg['sentiment'] = 0
print(gpt_neg['sentiment'].sum()) # 합은 0
gpt_neg[gpt_neg['sentiment'] == '부정']

0


,Unnamed: 0,original_review,original_reply,rating,gpt_review,gpt_reply,sentiment,score


In [138]:
# 사용할 컬럼만 가져오기
cols = gpt_neg.columns[4:7] # 사용할 컬럼(gpt_review, gpt_reply, sentiment)만 가져오기
train_gpt = gpt_neg[cols]
train_gpt

,gpt_review,gpt_reply,sentiment
0,김치찌개는 별로였어요. 약간 김치국 느낌이 나서 실망했어요. 고기도 별로였고.,안녕하세요. 김치찌개와 고기에 대한 실망을 드려 죄송합니다. 지속적인 품질 개선을 ...,0
1,양이 너무 많아서 배터질뻔했어요. 서비스도 별로였어요.,배터질뻔하셨다니 정말 안타깝습니다. 양에 대한 조정을 더 신경써야겠습니다. 불편을 ...,0
2,음식이 너무 싱겁고 부족해서 맛이 없었어요. 배달도 늦게 왔고 실망스러웠어요.,음식 맛과 배달 시간에 대한 실망을 드려 죄송합니다. 앞으로 더 신속하고 맛있는 음...,0
3,잡채는 짜다고 하지도 않았는데 맛이 진짜 짜요. 먹기 어려웠어요.,잡채가 짜다는 피드백을 반영하지 못해 죄송합니다. 더 맛있고 조리하기 쉽도록 개선하...,0
4,가격에 비해 음식의 양도 적고 맛도 별로였어요. 실망이었습니다.,가격과 맛에 대한 실망을 드려 죄송합니다. 더 다양한 메뉴와 향상된 품질로 보답하겠...,0
...,...,...,...
4396,서비스도 좋지 않았고 음식도 맛이 별로였어요. 실망했습니다,서비스와 음식의 맛 모두 아쉽게 느껴져 죄송합니다. 향후에는 만족할 수 있는 서비...,0
4397,배달이 많이 늦어서 실망했어요. 음식도 그다지 맛있지 않았어요,늦은 배달로 인해 실망을 드렸군요. 더 신속한 배달을 위해 노력하겠습니다. 또한 ...,0
4398,여러 번 시켜먹었는데 이번에는 별로였어요. 왠지 점점 맛이 없어지는 것 같아요,연이어 아쉬운 경험을 드려 정말 죄송합니다. 계속해서 음식의 맛을 향상시킬 수 있...,0
4399,음식이 맛이 없었어요. 다른 곳으로 시켜서 먹어야겠어요,맛이 없다니 정말 죄송합니다. 다른 가게의 음식을 주문하셔서 만족스러운 식사를 하...,0


## 데이터 정제

- 특이 패턴 텍스트(' , ') 제거

In [139]:
train_gpt['gpt_reply'][3640]

' , 고기가 신선하지 않아서 냄새가 나셨다니 정말 죄송합니다. 저희는 항상 청결과 신선한 재료로 요리를 준비하는 데 신경을 써왔는데, 미리 조치하도록 하겠습니다. , '

In [140]:
text = train_gpt['gpt_reply'][3640]

pattern = r'^\s*,\s*'
result = re.sub(pattern, '', text)

print(result)

고기가 신선하지 않아서 냄새가 나셨다니 정말 죄송합니다. 저희는 항상 청결과 신선한 재료로 요리를 준비하는 데 신경을 써왔는데, 미리 조치하도록 하겠습니다. , 


In [141]:
reply_list = []
for reply in train_gpt['gpt_reply']:
    if isinstance(reply, str):
        pat = r'^\s*,\s*'
        result = re.sub(pattern, '', reply)
        reply_list.append(result)
    else:
        reply_list.append("")

train_gpt['gpt_reply'] = reply_list
train_gpt.head()

,gpt_review,gpt_reply,sentiment
0,김치찌개는 별로였어요. 약간 김치국 느낌이 나서 실망했어요. 고기도 별로였고.,안녕하세요. 김치찌개와 고기에 대한 실망을 드려 죄송합니다. 지속적인 품질 개선을 ...,0
1,양이 너무 많아서 배터질뻔했어요. 서비스도 별로였어요.,배터질뻔하셨다니 정말 안타깝습니다. 양에 대한 조정을 더 신경써야겠습니다. 불편을 ...,0
2,음식이 너무 싱겁고 부족해서 맛이 없었어요. 배달도 늦게 왔고 실망스러웠어요.,음식 맛과 배달 시간에 대한 실망을 드려 죄송합니다. 앞으로 더 신속하고 맛있는 음...,0
3,잡채는 짜다고 하지도 않았는데 맛이 진짜 짜요. 먹기 어려웠어요.,잡채가 짜다는 피드백을 반영하지 못해 죄송합니다. 더 맛있고 조리하기 쉽도록 개선하...,0
4,가격에 비해 음식의 양도 적고 맛도 별로였어요. 실망이었습니다.,가격과 맛에 대한 실망을 드려 죄송합니다. 더 다양한 메뉴와 향상된 품질로 보답하겠...,0


In [142]:
train_gpt['gpt_reply'][3640]

'고기가 신선하지 않아서 냄새가 나셨다니 정말 죄송합니다. 저희는 항상 청결과 신선한 재료로 요리를 준비하는 데 신경을 써왔는데, 미리 조치하도록 하겠습니다. , '

- 특이 답변 제거

In [143]:
train_gpt[train_gpt['gpt_reply'].str.contains("사장님의 답변이 없습니다 ,")]

,gpt_review,gpt_reply,sentiment
3693,후기가 너무 좋아서~ 설마? 했는데 어마! 입니다 기본에 충실하며 MZ콜라보? 재...,"사장님의 답변이 없습니다 ,",0
3694,"너무맛잇어용. 잘먹겠습니다 맛이 별로였어요, 음식이 촉촉하지 않고 식어서 먹기가 ...","사장님의 답변이 없습니다 ,",0
3695,처음 주문 했는데 정말 맛있게 잘 먹었습니다 닭볶음탕 땡기면 당연히 재주문 각입니...,"사장님의 답변이 없습니다 ,",0
3696,"매번먹을때마다느끼지만 여기만한 맛집이없네요 맛이 별로였어요, 음식이 촉촉하지 않고...","사장님의 답변이 없습니다 ,",0
3697,"너무너무 맛있게 잘먹었습니다~ 맛이 별로였어요, 음식이 촉촉하지 않고 식어서 먹기...","사장님의 답변이 없습니다 ,",0
3698,지금껏 먹은 닭볶음탕중에 최고입니다~ 정말 맛있네요 자주 시켜먹을께요 맛이 별로였...,"사장님의 답변이 없습니다 ,",0
3699,"몇번째 주문인지 다른 배달 어플포함 여튼 엄청 맛있어욬 맛이 별로였어요, 음식이 ...","사장님의 답변이 없습니다 ,",0
3700,여기는 무조건 때되면 시켜먹어야되요 안그러면 금단 증상옴 계속 생각남 맛이 별로였...,"사장님의 답변이 없습니다 ,",0
3701,"배달도빠르고 양도넉넉하구 그래서 맛이 별로였어요, 음식이 촉촉하지 않고 식어서 먹...","사장님의 답변이 없습니다 ,",0
3702,오랜만의 주문이다. 항상 맛있게 먹고 있다. 맛이 변하지 않는다는게 참 좋다. 같...,"사장님의 답변이 없습니다 ,",0


In [144]:
train_gpt = train_gpt[~train_gpt['gpt_reply'].str.contains("사장님의 답변이 없습니다 ,")]
train_gpt

,gpt_review,gpt_reply,sentiment
0,김치찌개는 별로였어요. 약간 김치국 느낌이 나서 실망했어요. 고기도 별로였고.,안녕하세요. 김치찌개와 고기에 대한 실망을 드려 죄송합니다. 지속적인 품질 개선을 ...,0
1,양이 너무 많아서 배터질뻔했어요. 서비스도 별로였어요.,배터질뻔하셨다니 정말 안타깝습니다. 양에 대한 조정을 더 신경써야겠습니다. 불편을 ...,0
2,음식이 너무 싱겁고 부족해서 맛이 없었어요. 배달도 늦게 왔고 실망스러웠어요.,음식 맛과 배달 시간에 대한 실망을 드려 죄송합니다. 앞으로 더 신속하고 맛있는 음...,0
3,잡채는 짜다고 하지도 않았는데 맛이 진짜 짜요. 먹기 어려웠어요.,잡채가 짜다는 피드백을 반영하지 못해 죄송합니다. 더 맛있고 조리하기 쉽도록 개선하...,0
4,가격에 비해 음식의 양도 적고 맛도 별로였어요. 실망이었습니다.,가격과 맛에 대한 실망을 드려 죄송합니다. 더 다양한 메뉴와 향상된 품질로 보답하겠...,0
...,...,...,...
4396,서비스도 좋지 않았고 음식도 맛이 별로였어요. 실망했습니다,서비스와 음식의 맛 모두 아쉽게 느껴져 죄송합니다. 향후에는 만족할 수 있는 서비...,0
4397,배달이 많이 늦어서 실망했어요. 음식도 그다지 맛있지 않았어요,늦은 배달로 인해 실망을 드렸군요. 더 신속한 배달을 위해 노력하겠습니다. 또한 ...,0
4398,여러 번 시켜먹었는데 이번에는 별로였어요. 왠지 점점 맛이 없어지는 것 같아요,연이어 아쉬운 경험을 드려 정말 죄송합니다. 계속해서 음식의 맛을 향상시킬 수 있...,0
4399,음식이 맛이 없었어요. 다른 곳으로 시켜서 먹어야겠어요,맛이 없다니 정말 죄송합니다. 다른 가게의 음식을 주문하셔서 만족스러운 식사를 하...,0


In [145]:
train_gpt.iloc[3700]

gpt_review     음식이 촉촉하지 않고 맛이 없어서 별로였어요. 너무 실망스러웠어요. 다음에는 절대...
gpt_reply      맛이 부족했고 실망스러웠던 점 정말 죄송합니다. 다음번에는 더 맛있는 음식을 제공...
sentiment                                                     0
Name: 3710, dtype: object

In [146]:
len(train_gpt['gpt_review']) , len(train_gpt['gpt_reply'])

(4391, 4391)

- 공백으로 시작하는 텍스트 제거

In [147]:
reply_list = []
for reply in train_gpt['gpt_reply']:
    if isinstance(reply, str):
        pat = r'^\s*'
        result = re.sub(pat, '', reply)
        reply_list.append(result)
    else:
        reply_list.append("")

train_gpt['gpt_reply'] = reply_list
train_gpt.head()

,gpt_review,gpt_reply,sentiment
0,김치찌개는 별로였어요. 약간 김치국 느낌이 나서 실망했어요. 고기도 별로였고.,안녕하세요. 김치찌개와 고기에 대한 실망을 드려 죄송합니다. 지속적인 품질 개선을 ...,0
1,양이 너무 많아서 배터질뻔했어요. 서비스도 별로였어요.,배터질뻔하셨다니 정말 안타깝습니다. 양에 대한 조정을 더 신경써야겠습니다. 불편을 ...,0
2,음식이 너무 싱겁고 부족해서 맛이 없었어요. 배달도 늦게 왔고 실망스러웠어요.,음식 맛과 배달 시간에 대한 실망을 드려 죄송합니다. 앞으로 더 신속하고 맛있는 음...,0
3,잡채는 짜다고 하지도 않았는데 맛이 진짜 짜요. 먹기 어려웠어요.,잡채가 짜다는 피드백을 반영하지 못해 죄송합니다. 더 맛있고 조리하기 쉽도록 개선하...,0
4,가격에 비해 음식의 양도 적고 맛도 별로였어요. 실망이었습니다.,가격과 맛에 대한 실망을 드려 죄송합니다. 더 다양한 메뉴와 향상된 품질로 보답하겠...,0


In [148]:
review_list = []
for review in train_gpt['gpt_review']:
    if isinstance(review, str):
        pat = r'^\s*'
        result = re.sub(pat, '', review)
        review_list.append(result)
    else:
        review_list.append("")

train_gpt['gpt_review'] = review_list
train_gpt.head()

,gpt_review,gpt_reply,sentiment
0,김치찌개는 별로였어요. 약간 김치국 느낌이 나서 실망했어요. 고기도 별로였고.,안녕하세요. 김치찌개와 고기에 대한 실망을 드려 죄송합니다. 지속적인 품질 개선을 ...,0
1,양이 너무 많아서 배터질뻔했어요. 서비스도 별로였어요.,배터질뻔하셨다니 정말 안타깝습니다. 양에 대한 조정을 더 신경써야겠습니다. 불편을 ...,0
2,음식이 너무 싱겁고 부족해서 맛이 없었어요. 배달도 늦게 왔고 실망스러웠어요.,음식 맛과 배달 시간에 대한 실망을 드려 죄송합니다. 앞으로 더 신속하고 맛있는 음...,0
3,잡채는 짜다고 하지도 않았는데 맛이 진짜 짜요. 먹기 어려웠어요.,잡채가 짜다는 피드백을 반영하지 못해 죄송합니다. 더 맛있고 조리하기 쉽도록 개선하...,0
4,가격에 비해 음식의 양도 적고 맛도 별로였어요. 실망이었습니다.,가격과 맛에 대한 실망을 드려 죄송합니다. 더 다양한 메뉴와 향상된 품질로 보답하겠...,0


- 영어 답변 제거

In [149]:
train_gpt['gpt_reply'][2500]

'We re sorry to hear that the pizza arrived cold and tasteless. We value your feedback and will take immediate action to prevent such incidents from reoccurring. , '

In [150]:
train_gpt[train_gpt['gpt_reply'].str.match('^[a-zA-Z]')]

,gpt_review,gpt_reply,sentiment
2494,"맛이 별로였어요, 음식이 촉촉하지 않고 식어서 먹기가 좀 그랬네요. 다음에는 안시킬...","Dear customer, we apologize for the unpleasant...",0
2495,양이 너무 적고 서비스도 별로였어요.,Thank you for your feedback. We apologize for ...,0
2496,"Delicious, but the delivery was extremely slow.",We re sorry to hear that the delivery was slow...,0
2497,피자가 맛없고 느끼하고 배달이 너무 느렸어요,We apologize for the tasteless and heavy pizza...,0
2498,항상 실망시키네요. 다음에는 안시킬 거에요,We re sorry to have consistently disappointed ...,0
2499,맛이 좀 없어서 먹기가 그렇게 좋지 않았어요,We apologize for the lack of flavor in the foo...,0
2500,The pizza was cold and tasteless. I won t be o...,We re sorry to hear that the pizza arrived col...,0
2501,구성도 별로였고 배달도 엄청 늦었어요. 다신 시키지 않을 거에요,We apologize for the poor composition of the f...,0
2502,배달이 너무 느렸고 맛도 그냥 그랬어요,We apologize for the slow delivery and average...,0
2503,맛있지 않고 서비스도 별로였어요.,We re sorry to hear that our food and service ...,0


In [151]:
train_gpt = train_gpt[~train_gpt['gpt_reply'].str.match('^[a-zA-Z]')]
train_gpt

,gpt_review,gpt_reply,sentiment
0,김치찌개는 별로였어요. 약간 김치국 느낌이 나서 실망했어요. 고기도 별로였고.,안녕하세요. 김치찌개와 고기에 대한 실망을 드려 죄송합니다. 지속적인 품질 개선을 ...,0
1,양이 너무 많아서 배터질뻔했어요. 서비스도 별로였어요.,배터질뻔하셨다니 정말 안타깝습니다. 양에 대한 조정을 더 신경써야겠습니다. 불편을 ...,0
2,음식이 너무 싱겁고 부족해서 맛이 없었어요. 배달도 늦게 왔고 실망스러웠어요.,음식 맛과 배달 시간에 대한 실망을 드려 죄송합니다. 앞으로 더 신속하고 맛있는 음...,0
3,잡채는 짜다고 하지도 않았는데 맛이 진짜 짜요. 먹기 어려웠어요.,잡채가 짜다는 피드백을 반영하지 못해 죄송합니다. 더 맛있고 조리하기 쉽도록 개선하...,0
4,가격에 비해 음식의 양도 적고 맛도 별로였어요. 실망이었습니다.,가격과 맛에 대한 실망을 드려 죄송합니다. 더 다양한 메뉴와 향상된 품질로 보답하겠...,0
...,...,...,...
4396,서비스도 좋지 않았고 음식도 맛이 별로였어요. 실망했습니다,서비스와 음식의 맛 모두 아쉽게 느껴져 죄송합니다. 향후에는 만족할 수 있는 서비스...,0
4397,배달이 많이 늦어서 실망했어요. 음식도 그다지 맛있지 않았어요,늦은 배달로 인해 실망을 드렸군요. 더 신속한 배달을 위해 노력하겠습니다. 또한 음...,0
4398,여러 번 시켜먹었는데 이번에는 별로였어요. 왠지 점점 맛이 없어지는 것 같아요,연이어 아쉬운 경험을 드려 정말 죄송합니다. 계속해서 음식의 맛을 향상시킬 수 있도...,0
4399,음식이 맛이 없었어요. 다른 곳으로 시켜서 먹어야겠어요,맛이 없다니 정말 죄송합니다. 다른 가게의 음식을 주문하셔서 만족스러운 식사를 하실...,0


# 저장

In [152]:
train_gpt.to_csv(f'{DATA_PATH}train_gpt_4400.csv', index=False, encoding = "utf-8-sig")

In [27]:
# 엑셀로도 저장해서 확인해보기
with pd.ExcelWriter("train_gpt_4400.xlsx", engine='openpyxl') as writer:
    train_gpt.to_excel(writer, sheet_name='시트1', index=False)

- 불러오기

In [28]:
train_gpt = pd.read_csv(f'{DATA_PATH}train_gpt_4400.csv')
train_gpt

,gpt_review,gpt_reply,sentiment
0,김치찌개는 별로였어요. 약간 김치국 느낌이 나서 실망했어요. 고기도 별로였고.,안녕하세요. 김치찌개와 고기에 대한 실망을 드려 죄송합니다. 지속적인 품질 개선을 ...,0
1,양이 너무 많아서 배터질뻔했어요. 서비스도 별로였어요.,배터질뻔하셨다니 정말 안타깝습니다. 양에 대한 조정을 더 신경써야겠습니다. 불편을 ...,0
2,음식이 너무 싱겁고 부족해서 맛이 없었어요. 배달도 늦게 왔고 실망스러웠어요.,음식 맛과 배달 시간에 대한 실망을 드려 죄송합니다. 앞으로 더 신속하고 맛있는 음...,0
3,잡채는 짜다고 하지도 않았는데 맛이 진짜 짜요. 먹기 어려웠어요.,잡채가 짜다는 피드백을 반영하지 못해 죄송합니다. 더 맛있고 조리하기 쉽도록 개선하...,0
4,가격에 비해 음식의 양도 적고 맛도 별로였어요. 실망이었습니다.,가격과 맛에 대한 실망을 드려 죄송합니다. 더 다양한 메뉴와 향상된 품질로 보답하겠...,0
...,...,...,...
4396,서비스도 좋지 않았고 음식도 맛이 별로였어요. 실망했습니다,서비스와 음식의 맛 모두 아쉽게 느껴져 죄송합니다. 향후에는 만족할 수 있는 서비...,0
4397,배달이 많이 늦어서 실망했어요. 음식도 그다지 맛있지 않았어요,늦은 배달로 인해 실망을 드렸군요. 더 신속한 배달을 위해 노력하겠습니다. 또한 ...,0
4398,여러 번 시켜먹었는데 이번에는 별로였어요. 왠지 점점 맛이 없어지는 것 같아요,연이어 아쉬운 경험을 드려 정말 죄송합니다. 계속해서 음식의 맛을 향상시킬 수 있...,0
4399,음식이 맛이 없었어요. 다른 곳으로 시켜서 먹어야겠어요,맛이 없다니 정말 죄송합니다. 다른 가게의 음식을 주문하셔서 만족스러운 식사를 하...,0
